In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

In [2]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
books_data=books.merge(ratings,on="ISBN")
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [7]:
df=books_data.copy()
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0
...,...,...,...,...,...,...,...,...,...,...
1031131,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,276463,7
1031132,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,276579,4
1031133,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,276680,0
1031134,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,276680,0


In [8]:
df.dropna(inplace=True)

In [9]:
df.reset_index(drop=True,inplace=True)

In [10]:
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())

# Popularity based recommendation system

In [18]:
def popular_books(df, n=100):
    # Count the number of ratings for each book
    rating_count = df.pivot_table(index='Book-Title',values='Book-Rating',aggfunc='count').reset_index()
    rating_count.rename(columns={"Book-Rating": "NumberOfVotes"}, inplace=True)

    # Calculate the average rating for each book
    rating_average = df.pivot_table(index='Book-Title',values='Book-Rating',aggfunc='mean').reset_index()
    rating_average.rename(columns={"Book-Rating": "AverageRatings"}, inplace=True)

    # Merge the count and average DataFrames
    popularBooks = rating_count.merge(rating_average, on="Book-Title")

    # Define constants for the weighted rating formula
    C = popularBooks["AverageRatings"].mean()
    m = popularBooks["NumberOfVotes"].quantile(0.90)

    # Filter out books with fewer than 250 votes
    popularBooks = popularBooks[popularBooks["NumberOfVotes"] >= 250]

    # Define a function for the weighted rating calculation
    def weighted_rate(x):
        v = x["NumberOfVotes"]
        R = x["AverageRatings"]
        return ((v * R) + (m * C)) / (v + m)

    # Apply the weighted rating function to calculate popularity
    popularBooks["Popularity"] = popularBooks.apply(weighted_rate, axis=1)

    # Sort the DataFrame by Popularity in descending order
    popularBooks = popularBooks.sort_values(by="Popularity", ascending=False)

    # Return the top n popular books
    return popularBooks[["Book-Title", "NumberOfVotes", "AverageRatings", "Popularity"]].reset_index(drop=True).head(n)


In [19]:
top_ten=pd.DataFrame(popular_books(df,10))
top_ten

,Book-Title,NumberOfVotes,AverageRatings,Popularity
0,Harry Potter and the Prisoner of Azkaban Book 3,277,9.043321,9.016348
1,To Kill a Mockingbird,267,8.977528,8.950773
2,Harry Potter and the Sorcerer s Stone Harry Po...,315,8.936508,8.914407
3,Harry Potter and the Chamber of Secrets Book 2,326,8.840491,8.820575
4,Tuesdays with Morrie An Old Man a Young Man an...,250,8.588000,8.567099
5,The Secret Life of Bees,406,8.477833,8.466205
6,The Da Vinci Code,494,8.439271,8.430081
7,The Lovely Bones A Novel,707,8.185290,8.180632
8,The Red Tent Bestselling Backlist,383,8.182768,8.174253
9,Where the Heart Is Oprah s Book Club Paperback,295,8.142373,8.132034


# Item Based Collaborative Filtering

In [31]:
def item_based(bookTitle):
    # Convert the input to a string
    bookTitle = str(bookTitle)
    
    # Check if the bookTitle exists in the "Book-Title" column of the DataFrame (df)
    if bookTitle in df["Book-Title"].values:
        # Count the number of ratings for each book
        rating_count = pd.DataFrame(df["Book-Title"].value_counts())
        
        # Identify rare books with 200 or fewer ratings
        rare_books = rating_count[rating_count["Book-Title"] <= 200].index
        common_books = df[~df["Book-Title"].isin(rare_books)]
        
        # If the requested book is rare
        if bookTitle in rare_books:
            # Sample 3 most common books as recommendations
            most_common = pd.Series(common_books["Book-Title"].unique()).sample(3).values
            print("No Recommendations for this Book ☹️ \n ")
            print("YOU MAY TRY: \n ")
            print("{}".format(most_common[0]), "\n")
            print("{}".format(most_common[1]), "\n")
            print("{}".format(most_common[2]), "\n")
        else:
            # Create a pivot table for common books
            common_books_pivot = common_books.pivot_table(index=["User-ID"], columns=["Book-Title"], values="Book-Rating")
            title = common_books_pivot[bookTitle]
            
            # Calculate book correlations with the requested book
            recommendation_df = pd.DataFrame(common_books_pivot.corrwith(title).sort_values(ascending=False)).reset_index(drop=False)
            
            # Drop the row corresponding to the requested book
            if bookTitle in [title for title in recommendation_df["Book-Title"]]:
                recommendation_df = recommendation_df.drop(recommendation_df[recommendation_df["Book-Title"] == bookTitle].index[0])
            
            #print("recommendation_df")
            #print(recommendation_df)
            # Identify books with average ratings less than 5
            less_rating = [i for i in recommendation_df["Book-Title"] if df[df["Book-Title"] == i]["Book-Rating"].mean() < 5]
            
            # If there are more than 5 books with ratings greater than 5
            if recommendation_df.shape[0] - len(less_rating) > 5:
                recommendation_df = recommendation_df[~recommendation_df["Book-Title"].isin(less_rating)]
            
            # Select the top 5 recommendations
            recommendation_df = recommendation_df[0:5]
            recommendation_df.columns = ["Book-Title", "Correlation"]
            for i in recommendation_df["Book-Title"]:
                print(i,'with a rating of',round(df[df["Book-Title"]==i]["Book-Rating"].mean(),2))
    else:
        print("❌ COULD NOT FIND ❌")


In [32]:
item_based("Me Talk Pretty One Day")

The Poisonwood Bible A Novel with a rating of 8.18
The Chamber with a rating of 7.42
She s Come Undone Oprah s Book Club with a rating of 7.7
Timeline with a rating of 7.87
The Testament with a rating of 7.52


# User based Collaborative Filtering

In [39]:
x = df['User-ID'].value_counts() > 200
y = x[x].index
new_df = df[df['User-ID'].isin(y)]

In [40]:
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)

In [41]:
def users_choice(id):
    
    users_fav=new_df[new_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return users_fav

In [42]:
def user_based(new_df,id):
    if id not in new_df["User-ID"].values:
        print("❌ User NOT FOUND ❌")
        
        
    else:
        index=np.where(users_pivot.index==id)[0][0]
        similarity=cosine_similarity(users_pivot)
        similar_users=list(enumerate(similarity[index]))
        similar_users = sorted(similar_users,key = lambda x:x[1],reverse=True)[0:5]
    
        user_rec=[]
    
        for i in similar_users:
                data=df[df["User-ID"]==users_pivot.index[i[0]]]
                user_rec.extend(list(data.drop_duplicates("User-ID")["User-ID"].values))
        
    return user_rec

In [43]:
def common(new_df,user,user_id):
    x=new_df[new_df["User-ID"]==user_id]
    recommend_books=[]
    user=list(user)
    for i in user:
        y=new_df[(new_df["User-ID"]==i)]
        books=y.loc[~y["Book-Title"].isin(x["Book-Title"]),:]
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recommend_books.extend(books["Book-Title"].values)
        
    return recommend_books[0:5]

In [58]:
user_id=random.choice(new_df["User-ID"].values)
user_choice_df=pd.DataFrame(users_choice(user_id))
user_favorite=users_choice(user_id)
n=len(user_choice_df["Book-Title"].values)
print("🟦 USER: {} ".format(user_id))
print("\n")
    
print("Your favorite books:")
print("\n")    
for i in range(n):
    print(user_choice_df['Book-Title'].tolist()[i],"with an average rating of",round(new_df[new_df["Book-Title"]==user_choice_df["Book-Title"].tolist()[i]]["Book-Rating"].mean(),1))


user_based_rec=user_based(new_df,user_id)
books_for_user=common(new_df,user_based_rec,user_id)
books_for_userDF=pd.DataFrame(books_for_user,columns=["Book-Title"])
print("\n")
print("You may also like these books:")
print("\n")
for i in range(5):
    print(books_for_userDF['Book-Title'].tolist()[i],"with an average rating of",round(new_df[new_df["Book-Title"]==books_for_userDF["Book-Title"].tolist()[i]]["Book-Rating"].mean(),1))



🟦 USER: 88733 


Your favorite books:


The Good Red Road Passages into Native America with an average rating of 10.0
Dear Dad Letters from an Adult Child with an average rating of 10.0
Microsoft Office 2000 No Experience Required with an average rating of 10.0
Random House Dictionary with an average rating of 10.0
52 Ways to Make a Difference 52 Deck Series with an average rating of 10.0


You may also like these books:


Parable of the Sower with an average rating of 9.0
The Npr Interviews 1994 with an average rating of 10.0
The Far Side Observer with an average rating of 7.6
Wiener Dog Art with an average rating of 8.8
The Collected Stories of Eudora Welty with an average rating of 10.0
